# Managing Device Data (C/C++)

#### Sections
- [Learning Objectives](#Learning-Objectives)
- [Data Offload](#Data-Offload)
- [Map Clause](#Map-Clause)
- _Code:_ [Lab Exercise: Map Clause](#Lab-Exercise:-Map-Clause)
- [Dynamically Allocated Data and Length Specification](#Dynamically-Allocated-Data-and-Length-Specification)
- [Target Data Region](#Target-Data-Region)
- _Code:_ [Lab Exercise: Target Data Region](#Lab-Exercise:-Target-Data-Region)
- [Mapping Global Variables to Device](#Mapping-Global-Variables-to-Device)

## Learning Objectives
* Use OpenMP* constructs to effectively manage data transfers to and from the device 
* Be able to create a device data environment and map data to the device data environment
* Map global variables to OpenMP devices

### Prerequisites
Basic understanding of OpenMP constructs are assumed for this module. You also should have already went through the  [Introduction to OpenMP Offload module](../intro/intro.ipynb) where the basics of using the Jupyter notebooks with the Intel® devcloud and an introduction to the OpenMP `target` constructs were discussed.

***
## Data Offload
The host and devices have separate memory spaces, so when parts of the code are offloaded, data needs to be mapped to the target device in order to be accessed inside the target region.

By default, variables accessed inside the target region are treated as follows:

|Type | Behavior |
|:----:|:----|
|Scalars | Treated as `firstprivate` |
|Static arrays | Copied to the device on entry and from the device to the host on exit |
|Dynamic arrays | Same as above, length must be specified |

In the following example, the compiler will identify all variables that are used in the target region (a, x, and y), and data will be transferred to the device based on the above rules.

```c
void saxpy() {
    float a, x[ARRAY_SIZE], y[ARRAY_SIZE];
    #pragma omp target
    // On entry of target region, a, x, and y copied from host to the device
    for (int i=0; i< ARRAY_SIZE; i++) {
        y[i] = a * x[i] + y[i];
    }
    // Upon exit of the target region, both x and y are copied back to the host, 
    // even though x was not changed.
}
```

## Map Clause

To eliminate unnecessary data copies, use the `map` clause of the `target` directive to manually map variables to the device data environment.
```c
#pragma omp target map (map-type: list)
```
Available *map-type*s are
* `alloc`: Allocate storage for variable on target device, values not copied
* `to`: Allocate storage on target device and assign value **from original host variable to device** on target region entry
* `from`: Allocate storage on target device and assign value **from device to original host variable** on target region exit
* `tofrom`: default, both `to` and `from`

<img src="Assets/mapclause.jpg">

## Lab Exercise: Map Clause

In this exercise you will add a map clause to the saxpy ($y=a*x+y$) operation. The primary source file, main.cpp, is written for you. It includes saxpy_func.cpp that you will complete and write out to file in this Jupyter notebook. If you would like to see the contents of main.cpp, execute the following cell.


In [1]:
#Optional, see the contents of main.cpp
%pycat main.cpp

//==============================================================
// Copyright © 2020 Intel Corporation
//
// SPDX-License-Identifier: MIT
// =============================================================

#include <omp.h>
#include <stdio.h>

constexpr int ARRAY_SIZE = 256;

int main(int argc, char *argv[]) {
  int i, j, is_cpu = true;
  double tstart, tstop;
  float x[ARRAY_SIZE], y[ARRAY_SIZE];
  float a = 1.5f;
  float tolerance = 0.01f;
  int correct_count = 0;

  // Initialize some data
  for (i = 0; i < ARRAY_SIZE; i++) {
    x[i] = (float)i;
    y[i] = (float)i;
  }

  tstart = omp_get_wtime();

#include "lab/saxpy_func.cpp"

  tstop = omp_get_wtime();
  printf("Work took %f seconds\n", tstop - tstart);

  printf("Running on %s.\n", is_cpu ? "CPU" : "GPU");

  for (int i = 0; i < ARRAY_SIZE; i++)
    if (abs(y[i] - (a * i + i)) < tolerance)
      correct_count++;
    else {
      printf("Incorrect Result at Element [%d] : %f", i, y[i]);
      break;
    }
  printf("Test: %s\n", (c

In the cell below, add the map clause that would map the x array to the target so that it won't be unnecessarily copied back. Also, add the clause `map(from:is_cpu)` so we'll know whether the code was executed on the GPU.

In [2]:
%%writefile lab/saxpy_func.cpp
// Add the target pragma with the map clauses here
#pragma omp target map(to:x[0:ARRAY_SIZE]) map(tofrom:y[0:ARRAY_SIZE]) map(from:is_cpu)
{
  is_cpu = omp_is_initial_device();
  for (i = 0; i < ARRAY_SIZE; i++) {
    y[i] = a * x[i] + y[i];
  }
}

Overwriting lab/saxpy_func.cpp


### Compile and Execute the Code
Next, compile and execute the code using *run.sh*. If you would like to see the contents of run.sh execute the following cell.

In [3]:
# Optional: Run this cell to see the contents of run.sh
%pycat run.sh

#!/bin/bash
source /opt/intel/oneapi/setvars.sh > /dev/null 2>&1
/bin/echo "##" $(whoami) is running OMP_Offload Module2 -- Data Transfer - 1 of 2 main.cpp/main.f90
echo "########## Compiling"
# export LIBOMTARGET_DEBUG=1; 
icpx -qopenmp -fopenmp-targets=spir64 main.cpp -o bin/a.out || exit $?
echo "########## Executing"
cd bin
./a.out
echo "########## Done"


Execute the following cell to execute main.cpp. Look for the PASSED! message.

_If the Jupyter cells are not responsive or if they error out when you compile the samples, please restart the Kernel and compile the samples again_

In [4]:
! chmod 755 q; chmod 755 run.sh;if [ -x "$(command -v qsub)" ]; then ./q run.sh; else ./run.sh; fi

## ubeb158842a2b8e63a23d31935e35063 is running OMP_Offload Module2 -- Data Transfer - 1 of 2 main.cpp/main.f90
########## Compiling
########## Executing
Work took 0.549738 seconds
Running on GPU.
Test: PASSED!
########## Done


Execute the following cell to see the solution.

In [5]:
%pycat saxpy_func_solution.cpp

// Add the target pragma with the map clauses here
#pragma omp target map(tofrom : y) map(to : x) map(from : is_cpu)
{
  is_cpu = omp_is_initial_device();
  for (i = 0; i < ARRAY_SIZE; i++) {
    y[i] = a * x[i] + y[i];
  }
}


## Dynamically Allocated Data and Length Specification

For dynamically allocated arrays, when using the `target map` construct, the number of elements to be mapped must be explicitly specified. Partial arrays maybe specified.
```c
#pragma omp target map(to:array[start:length])
```
In the previous example, x and y are static arrays, so length specification is optional. If you wish you may go back to the previous example and specify the size of the array to map. Alternatively, you may run the following cell to see the solution.

In [6]:
%pycat saxpy_func_solution_length.cpp

// Add the target pragma with the map clauses here
#pragma omp target map(tofrom: y[0:ARRAY_SIZE]) map(to: x[0:ARRAY_SIZE]) map(from: is_cpu)
{
    is_cpu = omp_is_initial_device();
    for (i = 0; i < ARRAY_SIZE; i++) {
        y[i] = a * x[i] + y[i];
    }
}


***
## Target Data Region
When there are more than one target regions, it's often useful to create a larger target **data** region that encompasses all of the target regions to minimize data copy across target regions. There are two ways to create a target data region, using `target data` or using `target enter data` and `target exit data`.
### Target Data
The `target data` construct creates a scoped data environment and maps data to and from the device. When using this construct, the `alloc`, `to`, `from`, and `tofrom` map-types are available. 

Note: `Target Data` does not create a target region that offloads execution. `target` constructs inside the data environment is needed to accomplishes that.
```c
#pragma omp target data map(tofrom: x)
// Device data environment created, x stays on the device through out the two target regions
{
    #pragma omp target(to: y)
    {
        // First target region
    }
    host_update(y); // y must be mapped at each target region because it's being updated by the host
    #pragma omp target(to: y)
    {
        // Second target region
    }
}
```
### Target Enter/Exit Data and Update
`target enter/exit data` constructs can be used to explicitly mark the beginning and ending of the target data environment.

When using the `target enter data` construct, only the map-types of `alloc` and `to` are available. When using the `target exit data` construct, the `from`, `release`, and `delete` map-types are available. 

The `target update` construct is used to issue data transfers to or from the existing data device environment.

Note: `target enter/exit/update data` constructs are not scoped and does not offload execution of code. `target` constructs are needed between enter and exit of data environment to accomplish that.

Example:
```c
#pragma omp target enter data map(to:y) map(alloc: x)
#pragma omp target
{ //First target region, device operations on x and y
}

#pragma omp target update from (y)
host_update(y);
#pragma omp target update to (y)

#pragma omp target
{ //2nd target region, device operations on x and y
}
#pragma omp target exit data map(from:x)
```

## Lab Exercise: Target Data Region
In this exercise, we have two target regions. x and y are static arrays of size ARRAY_SIZE, and they are used in the target regions. In addition, the value of y is updated by the host between the regions. For this program, *main_data_region.cpp* contains main and includes *target_data_region.cpp*, which is the file you will override.

Create a target data environment that encompasses both target regions, ensure `x` stays on the device across the region and make sure `y` is updated to the device after the host `init2` call. Test your code, and ensure the PASSED message is displayed.

There are two ways to solve this problem. You may choose to use either `target data` or `target enter/update/exit data`. Solution is provided for both.

In [7]:
#Examine main_data_region.cpp if you wish.
%pycat main_data_region.cpp

//==============================================================
// Copyright © 2020 Intel Corporation
//
// SPDX-License-Identifier: MIT
// =============================================================

#include <omp.h>
#include <stdio.h>

constexpr int ARRAY_SIZE = 256;

void init1(float *x, int N) {
  for (int i = 0; i < N; i++) x[i] = 1.0;
}
void init2(float *x, int N) {
  for (int i = 0; i < N; i++) x[i] = 2.0;
}
int main() {
  float x[ARRAY_SIZE], y[ARRAY_SIZE];
  double tb, te;
  int correct_count = 0;

  init1(x, ARRAY_SIZE);
  init1(y, ARRAY_SIZE);

  printf("Number of OpenMP Devices: %d\n", omp_get_num_devices());

  tb = omp_get_wtime();

#include "lab/target_data_region.cpp"

  te = omp_get_wtime();

  printf("Time of kernel: %lf seconds\n", te - tb);

  for (int i = 0; i < ARRAY_SIZE; i++)
    if (x[i] == 4.0) correct_count++;

  printf("Test: %s\n", (correct_count == ARRAY_SIZE) ? "PASSED!" : "Failed");
}


In [8]:
%%writefile lab/target_data_region.cpp
#pragma omp target enter data map(to : x) map(to : y)
#pragma omp target 
  {
    for (int i = 0; i < ARRAY_SIZE; i++) x[i] += y[i];
  }
  init2(y, ARRAY_SIZE);
#pragma omp target update to(y)

#pragma omp target
  {
    for (int i = 0; i < ARRAY_SIZE; i++) x[i] += y[i];
  }
#pragma omp target exit data map(from:x) 

Overwriting lab/target_data_region.cpp


### Compile and Execute the Code

In [9]:
# Optional: Examine the run script if you choose
%pycat run_data.sh

#!/bin/bash
source /opt/intel/oneapi/setvars.sh > /dev/null 2>&1
/bin/echo "##" $(whoami) is running OMP_Offload Module2 -- Data Transfer - 2 of 2 main_data_region.cpp/f90
echo "########## Compiling"
icpx -qopenmp -fopenmp-targets=spir64 main_data_region.cpp -o bin/data_region.out || exit $?
echo "########## Executing"
cd bin
./data_region.out
echo "########## Done"


In [10]:
#Execute the program, if you see the "FAILED" message, go back and debug your code
! chmod 755 q; chmod 755 run_data.sh;if [ -x "$(command -v qsub)" ]; then ./q run_data.sh; else ./run_data.sh; fi

## ubeb158842a2b8e63a23d31935e35063 is running OMP_Offload Module2 -- Data Transfer - 2 of 2 main_data_region.cpp/f90
########## Compiling
########## Executing
Number of OpenMP Devices: 4
Time of kernel: 0.724958 seconds
Test: PASSED!
########## Done


_If the Jupyter cells are not responsive or if they error out when you compile the samples, please restart the Kernel and compile the samples again_

In [11]:
#Examine both solutions
%pycat target_data_region_solution.cpp

// Solution Using Target Data
#pragma omp target data map(tofrom : x)
{
#pragma omp target map(to : y)
  {
    for (int i = 0; i < ARRAY_SIZE; i++) x[i] += y[i];
  }

  init2(y, ARRAY_SIZE);

#pragma omp target map(to : y)
  {
    for (int i = 0; i < ARRAY_SIZE; i++) x[i] += y[i];
  }
}


// Solution Using Target Enter/Exit/Update
#pragma omp target enter data map(to : x) map(to : y)
#pragma omp target
{
  for (int i = 0; i < ARRAY_SIZE; i++) x[i] += y[i];
}

init2(y, ARRAY_SIZE);

#pragma omp target update to(y)

#pragma omp target
{
  for (int i = 0; i < ARRAY_SIZE; i++) x[i] += y[i];
}
#pragma omp target exit data map(from : x)


## Mapping Global Variables to Device
With OpenMP, you also have the option to map a variable to the device for the duration of the program. Use the `declare target` directive to specify that variables and functions are mapped to a device. Here's an example.
```c
#pragma omp declare target
int a[N]
#pragma omp end declare target

...
    
//Host Code
init(a);

#pragma omp target
for (int i=0; i<N; i++) {
    result[i]=process(a[i]);
}
```

# Summary
In this module, you have learned the following:
* How OpenMP handles data transfers to the device by default
* Explicitly specify data mapping in the `#pragma omp target` construct with the map clause
* Declare target data region with `target data` and `target enter/exit data` constructs
* Explicitly issue data transfers using the `target update` directive
* Map global variables to the target device

<html><body><span style="color:green"><h1>Survey</h1></span></body></html>

[Tell us how we did in this module with a short survey. We will use your feedback to improve the quality and impact of these learning materials. Thanks!](https://intel.az1.qualtrics.com/jfe/form/SV_1HsuWWKNt2BqWt7)

<html><body><span style="color:Red"><h1>Reset Notebook</h1></span></body></html>

##### Should you be experiencing any issues with your notebook or just want to start fresh run the below cell.

In [12]:
from IPython.display import display, Markdown, clear_output
import ipywidgets as widgets
button = widgets.Button(
    description='Reset Notebook',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='This will update this notebook, overwriting any changes.',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)
out = widgets.Output()
def on_button_clicked(_):
      # "linking function with output"
      with out:
          # what happens when we press the button
          clear_output()
          !rsync -a --size-only /data/oneapi_workshop/OpenMP_Offload/datatransfer/ ~/OpenMP_Offload/datatransfer
          print('Notebook reset -- now click reload on browser.')
# linking button and function together using a button's method
button.on_click(on_button_clicked)
# displaying button and its output together
widgets.VBox([button,out])

***

@Intel Corporation | [\*Trademark](https://www.intel.com/content/www/us/en/legal/trademarks.html)